In [1]:
!pip install pygbif
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dash 2.14.2 requires dash-core-components==2.0.0, which is not installed.
dash 2.14.2 

In [1]:
from pygbif import occurrences
import requests
from shapely.geometry import shape
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import time
import zipfile


kalkalpen_geojson = "https://raw.githubusercontent.com/stopopol/bmd_kalkalpen_case_study/" \
"refs/heads/main/input_data/np-kalkalpen_bounds_2025-12-22_bb.geojson"

zoebelboden_geojson = "https://raw.githubusercontent.com/stopopol/bmd_kalkalpen_case_study/" \
"refs/heads/main/input_data/zoebelboden_bounds_2025-12-22_bb.geojson"

# Load kalkalpen bounding box from URL
response = requests.get(kalkalpen_geojson)
response.raise_for_status()
geojson = response.json()

# Convert geometry to WKT string
geom = shape(geojson["features"][0]["geometry"])
geometry_wkt = geom.wkt

INFO:NumExpr defaulting to 4 threads.


In [4]:
res = occurrences.search(
    geometry=geometry_wkt,
    hasCoordinate=True,
    limit=0
)

total = res["count"]
print(total + " records within provided extent")

if total > 50000:
    print("Too many records — use GBIF download API")
else:
    print(f"{total} records — safe to paginate search API")

Too many records — use GBIF download API


In [23]:
import requests
import time
import zipfile
import pandas as pd
import geopandas as gpd

# GBIF credentials
auth = {
    "username": "****",
    "password": "*****",
    "email": "*****"
}

# Download request
download_request = {
    "creator": auth["username"],
    "notification_address": [auth["email"]],
    "predicate": {
        "type": "and",
        "predicates": [
            {
                "type": "within",
                "geometry": geometry_wkt
            }
        ]
    }
}

# Submit request
url = "https://api.gbif.org/v1/occurrence/download/request"
response = requests.post(url, auth=(auth["username"], auth["password"]), json=download_request)

if response.status_code == 201:
    download_key = response.text
    print("Download request submitted successfully! Key:", download_key)
    
    # Wait for download
    status_url = f"https://api.gbif.org/v1/occurrence/download/{download_key}"
    print("Current Time:", time.strftime("%H:%M:%S"))
    
    while True:
        r = requests.get(status_url, auth=(auth["username"], auth["password"]))
        data = r.json()
        status = data.get('status')
        if status == 'SUCCEEDED':
            print("Download ready!")
            break
        elif status in ['KILLED', 'FAILED']:
            raise Exception(f"Download failed: {data}")
        else:
            print("Waiting for download to complete...")
            time.sleep(60)
    
    # Download ZIP
    download_url = f"https://api.gbif.org/v1/occurrence/download/request/{download_key}.zip"
    zip_path = "gbif_download.zip"
    with requests.get(download_url, auth=(auth["username"], auth["password"]), stream=True) as r:
        with open(zip_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    print("Download saved as gbif_download.zip")
    
else:
    print("Failed to submit download request")
    print(response.status_code, response.text)

Download request submitted successfully! Key: 0063995-251120083545085
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to complete...
Waiting for download to comple

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR:sqlite3_exec(DETACH DATABASE "temp_rtree_34f6c3d68dc5fe7251e2b583829c5bde") failed: database temp_rtree_34f6c3d68dc5fe7251e2b583829c5bde is locked


CPLE_AppDefinedError: b'sqlite3_exec(DETACH DATABASE "temp_rtree_34f6c3d68dc5fe7251e2b583829c5bde") failed: database temp_rtree_34f6c3d68dc5fe7251e2b583829c5bde is locked'

In [1]:
# Convert to GeoPackage
with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open("occurrence.txt") as f:
        df = pd.read_csv(f, sep='\t', low_memory=False)
df = df.dropna(subset=['decimalLongitude', 'decimalLatitude'])
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['decimalLongitude'], df['decimalLatitude']),
    crs="EPSG:4326"
)
gdf.to_file("gbif_data.gpkg", layer="occurrences", driver="GPKG")
print("Saved as gbif_data.gpkg")

NameError: name 'zipfile' is not defined